In [ ]:
import requests
import base64

In [ ]:
# Replace with your own Client ID and Client Secret
CLIENT_ID = ''CLIENT_SECRET = ''
# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print('Access token obtained successfully')
else:
    print("Error obtaining access token.")
    exit()

access_token

In [ ]:
%pip install spotipy

In [ ]:
import pandas as pd
import spotipy

In [ ]:
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
def get_trending_playlist_data(playlist_id, acccess_token):
    #Setup Spotify with the access token
    sp = spotipy.Spotify(auth=access_token)

    #Get the tracks from playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []

    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        #Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        #Get the release date of album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        
        except:
            release_date = None

        #Get the popularity of tracks
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [ ]:

playlist_id = '6ZYZgxQxoyCGOrJsMAIR5v'

# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
print(music_df)

In [ ]:
print(music_df.isnull().sum())

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
data = music_df

In [ ]:
# Function to calculate weighted popularity scores based on release date
def calculate_weighted_popularity(release_date):
    # Convert the release date to datetime object
    release_date = datetime.strptime(release_date,'%Y-%m-%d')

    # Calculate the timespan between the release date and today's date
    time_span = datetime.now() - release_date

    # Calculate the weighted popularity score based on time span (e.g., more recent releases have higher weight)
    weight = 1 / (time_span.days + 1)
    return weight

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key','Loudness', 'Mode', 'Speechiness', 'Acousticness','Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

In [86]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    # Validate input song name
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Validate num_recommendations and alpha (customize the ranges as needed)
    if not (isinstance(num_recommendations, int) and num_recommendations > 0):
        print("Please enter a valid positive integer for num_recommendations.")
        return

    if not (0 <= alpha <= 1):
        print("Please enter a valid value for alpha in the range [0, 1].")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the input song row
    input_song_row = music_df[music_df['Track Name'] == input_song_name].iloc[0]

    # Get the popularity score of the input song
    popularity_score = input_song_row['Popularity']

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(input_song_row['Release Date'])

    # Assuming content_based_rec is a DataFrame generated from content-based recommendation logic
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Initialize hybrid_recommendations as an empty DataFrame
    hybrid_recommendations = pd.DataFrame()

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = pd.concat([content_based_rec, pd.DataFrame({
    'Track Name': [input_song_name],
    'Artists': [input_song_row['Artists']],
    'Album Name': [input_song_row['Album Name']],
    'Release Date': [input_song_row['Release Date']],
    'Popularity': [weighted_popularity_score]
    })], ignore_index=True)
    

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]

    return hybrid_recommendations

In [90]:
input_song_name = "Vaadiamma"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
print(recommendations)

Hybrid recommended songs for 'Vaadiamma':
               Track Name                                   Artists  \
4     Thanni Karuthirichi     Achu, Silambarasan TR, Vineetha Ajith   
3      Sarakku Vachiruken                Shankar Mahadevan, Radhika   
1          Rangu Rangamma  Vijay Yesudas, Kailash Kher, Swarnalatha   
2         Bothai Eripochu  Vidyasagar, S. P. Sailaja, Suresh Peters   
0  Neruppu Koothadikkuthu           Venkat Prabhu, Chitra Sivaraman   

                                          Album Name Release Date  Popularity  
4  Yennai Theriyuma (Original Motion Picture Soun...   2008-11-20        58.0  
3                                          Shajahaan   2001-05-09        52.0  
1        Bheema (Original Motion Picture Soundtrack)   2013-04-09        49.0  
2       Jaihind (Original Motion Picture Soundtrack)   1994-01-29        46.0  
0  Thulluvadho Ilamai (Original Motion Picture So...   2001-12-31        30.0  
